For the computation of our gradient we need to have the adjoint of the inverse wavelet transform. Usually the adjoint of the inverse wavelet transform is the wavelet transform itself. That is, we have:
$$
\psi^* = \psi^{-1}
$$

The definition of the adjoint $T^*$ of an operator $T$ defined from $E \rightarrow F$ is:
$$
<T(x); y>_F = <x; T^*(y)>_E \text{, } \forall x,y \in (E, F)
$$

We show here that this property is not verified with the implementations of the wavelet transform and the inverse wavelet transform in `pywavelet`, with the padding mode `"symmetric"`. We also show that the inverse wavelet transform is not the inverse of the wavelet transform if we consider all $R^p$ where $p$ is the number of wavelet coefficients.

However, with the padding mode `"periodization"`, the inverse wavelet transform is indeed the inverse (and therefore the adjoint) of the wavelet transform. (Even weirder, for zero-padding -padding mode `"zero"`-, the adjoint property is verified but not the inverse property).

The reason this was not spotted earlier is because we always assert the inverse in one direction not the other (therefore thinking that if the inverse property is verified, it must be the adjoint).

In [1]:
import numpy as np
import pywt

Define the wavelet transform and inverse transform

In [2]:
wavelet_type = "db4"
level = 4
mode = "zero"

coeffs_tpl = pywt.wavedecn(data=np.zeros((512, 512)), wavelet=wavelet_type, mode=mode, level=level)
coeffs_1d, coeff_slices, coeff_shapes = pywt.ravel_coeffs(coeffs_tpl)
coeffs_tpl_rec = pywt.unravel_coeffs(coeffs_1d, coeff_slices, coeff_shapes)
_ = pywt.waverecn(coeffs_tpl_rec, wavelet=wavelet_type, mode=mode)

def py_W(im):
    alpha = pywt.wavedecn(data=im, wavelet=wavelet_type, mode=mode, level=level)
    alpha, _, _ = pywt.ravel_coeffs(alpha)
    return alpha
    
def py_Ws(alpha):
    coeffs = pywt.unravel_coeffs(alpha, coeff_slices, coeff_shapes)
    im = pywt.waverecn(coeffs, wavelet=wavelet_type, mode=mode)
    return im

In [3]:
if coeffs_1d.shape[0] > 512 * 512:
    print("Wavelet is redundant and therefore not orthogonal")
else:
    print("Wavelet is orthogonal")

Wavelet is redundant and therefore not orthogonal


Test the adjoint and inverse properties (here in complex). Run this several times to have different random images and coefficients.

In [4]:
x_example = np.random.rand(*coeffs_1d.shape) + 1j * np.random.rand(*coeffs_1d.shape)
y_example = np.random.rand(512, 512)
print("Adjoint:")
x_Tadj_y = np.dot(x_example, np.conjugate(py_W(y_example)))
T_x_y = np.dot(py_Ws(x_example).flatten(), np.conjugate(y_example.flatten()))
print(np.allclose(x_Tadj_y, T_x_y))
print("\n Inverse from image to image:")
print(np.allclose(py_Ws(py_W(y_example)), y_example))
print("\n Inverse from coefficients to coefficients:")
print(np.allclose(py_W(py_Ws(x_example)), x_example))

Adjoint:
True

 Inverse from image to image:
True

 Inverse from coefficients to coefficients:
False
